# Proving the Playpen Agent/CodePlan for Compiler Errors

We need prove that the playpen code plan agents workflow will work as expected. This will be a way to run them and make sure that things work as we expect.

## Setup

In [1]:
# Need to make sure that XML tree can use the new parser
import sys

sys.modules['_elementtree'] = None
import pprint

pp = pprint.PrettyPrinter(indent=2)

## Create the Test Environment

For creating the test environment we will:

1. move the coolstore to a temp direcotry
2. Update the InventoryEntity to use UUID for the itemID as the type.

In [2]:
import tempfile
import shutil
import os
import subprocess

#First we need to download coolstore if it does not exist.

if not os.path.exists("./coolstore"):
    subprocess.run("../../example/fetch.sh")

if not os.path.exists("../../kai-analyzer/kai-analyzer"):
    subprocess.run(["go", "build", "-o", "kai-analyzer", "main.go"], cwd="../../kai-analyzer")

# NOTE(JonahSussman): Python's default tmp dir gets clobbered somehow on my
# machine, so putting it in local directory for now.
temp_dir = tempfile.TemporaryDirectory(prefix="tmp-", dir=os.getcwd())
coolstore_path = os.path.join(temp_dir.name, "coolstore")
shutil.copytree("./coolstore", coolstore_path)

## make the necessary change

print(temp_dir)


<TemporaryDirectory '/home/jonah/Projects/github.com/konveyor-ecosystem/kai-jonah/notebooks/kai-analyzer-code-plan/tmp-o0ipu7tj'>


## Set up the Code Plan types and run code plan.

In [3]:
from pathlib import Path
from kai.repo_level_awareness.api import RpcClientConfig
from kai.repo_level_awareness.codeplan import TaskManager
from kai.repo_level_awareness.task_runner.analyzer_lsp.validator import AnalyzerLSPStep, AnalyzerLSPStep
from kai.repo_level_awareness.task_runner.analyzer_lsp.task_runner import AnalyzerTaskRunner
from kai.repo_level_awareness.task_runner.compiler.maven_validator import MavenCompileStep
from kai.repo_level_awareness.task_runner.compiler.compiler_task_runner import MavenCompilerTaskRunner
from kai.repo_level_awareness.task_runner.dependency.task_runner import DependencyTaskRunner
from kai.repo_level_awareness.agent.dependency_agent.dependency_agent import MavenDependencyAgent
from kai.models.report_types import Incident, RuleSet, Violation, Category
from kai.service.llm_interfacing.model_provider import ModelProvider
from kai.models.kai_config import KaiConfig
from kai.repo_level_awareness.vfs.git_vfs import RepoContextManager
import logging
from kai.repo_level_awareness.task_runner.analyzer_lsp.api import AnalyzerDependencyRuleViolation, AnalyzerRuleViolation

logging.basicConfig(level=logging.DEBUG)
logging.getLogger('httpx').setLevel(logging.WARNING)
logging.getLogger('httpcore').setLevel(logging.WARNING)
logging.getLogger('genai.extensions.langchain.chat_llm').setLevel(logging.WARNING)

config = RpcClientConfig(Path(coolstore_path),
                         "../../kai-analyzer/kai-analyzer",
                         "/Users/shurley/repos/MTA/rulesets/default/generated",
                         Path("/Users/shurley/repos/kai/jdtls/bin/jdtls"),
                         Path("./java-bundle/java-analyzer-bundle.core-1.0.0-SNAPSHOT.jar"),
                         "konveyor.io/target=quarkus || konveyor.io/target=jakarta-ee",
                         None,
                         None)

# config = RpcClientConfig(Path(coolstore_path),
#                          "../../kai-analyzer/kai-analyzer",
#                          Path("/home/jonah/Projects/github.com/konveyor/rulesets/default/generated"),
#                          Path("/home/jonah/.vscode/extensions/redhat.java-1.35.1-linux-x64/server/bin/jdtls"),
#                          Path("./java-bundle/java-analyzer-bundle.core-1.0.0-SNAPSHOT.jar"),
#                          "konveyor.io/target=quarkus || konveyor.io/target=jakarta-ee",
#                          None,
#                          None)

kai_config = KaiConfig.model_validate_filepath("../../kai/config.toml")
modelProvider = ModelProvider(kai_config.models)
rcm = RepoContextManager(config.repo_directory, modelProvider.llm)

maven_dependency_agent = MavenDependencyAgent(modelProvider.llm, config.repo_directory)


anayzer_task_runner= AnalyzerTaskRunner(modelProvider.llm)
maven_compiler_task_runner= MavenCompilerTaskRunner(modelProvider.llm)
dependency_task_runner = DependencyTaskRunner(maven_dependency_agent)

# Define the initial task, to prove out the solving of a single incident:

path = os.path.join(coolstore_path, 'src/main/java/com/redhat/coolstore/service/ShippingService.java')
print(path)

incident = Incident(
    uri='file://'+path,
    message='Remote EJBs are not supported in Quarkus, and therefore its use must be removed and replaced with REST functionality. In order to do this:\n 1. Replace the `@Remote` annotation on the class with a `@jakarta.ws.rs.Path("<endpoint>")` annotation. An endpoint must be added to the annotation in place of `<endpoint>` to specify the actual path to the REST service.\n 2. Remove `@Stateless` annotations if present. Given that REST services are stateless by nature, it makes it unnecessary.\n 3. For every public method on the EJB being converted, do the following:\n - In case the method has no input parameters, annotate the method with `@jakarta.ws.rs.GET`; otherwise annotate it with `@jakarta.ws.rs.POST` instead.\n - Annotate the method with `@jakarta.ws.rs.Path("<endpoint>")` and give it a proper endpoint path. As a rule of thumb, the method name can be used as endpoint, for instance:\n ```\n @Path("/increment")\n public void increment() \n ```\n - Add `@jakarta.ws.rs.QueryParam("<param-name>")` to any method parameters if needed, where `<param-name>` is a name for the parameter.',
    code_snip=' 2  \n 3  import java.math.BigDecimal;\n 4  import java.math.RoundingMode;\n 5  \n 6  import javax.ejb.Remote;\n 7  import javax.ejb.Stateless;\n 8  \n 9  import com.redhat.coolstore.model.ShoppingCart;\n10  \n11  @Stateless\n12  @Remote\n13  public class ShippingService implements ShippingServiceRemote {\n14  \n15      @Override\n16      public double calculateShipping(ShoppingCart sc) {\n17  \n18          if (sc != null) {\n19  \n20              if (sc.getCartItemTotal() >= 0 && sc.getCartItemTotal() < 25) {\n21  \n22                  return 2.99;',
    line_number=12,
    variables={'file': 'file:///private/var/folders/vt/5bfp7vyd1h79_7k5ygr0fttr0000gn/T/tmpthgg63up/coolstore/src/main/java/com/redhat/coolstore/service/ShippingService.java', 'kind': 'Class', 'name': 'Stateless', 'package': 'com.redhat.coolstore.service'},
)

ruleset = RuleSet(
    name='quarkus/springboot',
    description='This ruleset gives hints to migrate from SpringBoot devtools to Quarkus',
    tags=None,
    violations={},
    errors=None,
    unmatched=None,
    skipped=None,
)

violation = Violation(
    description='Remote EJBs are not supported in Quarkus',
    category=Category.MANDATORY,
    labels=['konveyor.io/source=java-ee', 'konveyor.io/source=jakarta-ee', 'konveyor.io/target=quarkus'],
)

seed_task = AnalyzerRuleViolation(
    file=path,
    line=incident.line_number,
    column=None,
    message=incident.message,
    incident=incident,
    violation=violation,
    ruleset=ruleset,
)

# TODO: Use seed_tasks argument to supply initial task to the task_manager
task_manager = TaskManager(
        config,
        rcm,
        [seed_task],
        # TODO: Set up with maven as well?
        validators=[AnalyzerLSPStep(config), MavenCompileStep(config)],
        # Agents are really task_runners
        agents=[anayzer_task_runner, maven_compiler_task_runner, dependency_task_runner],
    )


# TODO: Make this get_next_task(max_priority=0) to only grab seeded tasks
# Can also do: 
#   initial_task = task_manager.get_next_task(max_priority=0)
#   ...
#   task_manager.supply_result(task_manager.execute_task(initial_task)
#   follow_up_task = task_manager.get_next_task(max_priority=0)
#   # do whatever to show what followup is
#   task_manager.supply_result(task_manager.execute_task(follow_up_task)
# etc ...
# can  introspect the stack using task_manager.task_stacks
# priority 0 tasks will be accessible with task_manager.task_stacks.get(0)
# So can see all the new tasks that are spawned and how that stack changes as we progress
# as well as showing all the tasks in the stack if we want to show all the work that's been detected in general
# Can do that with task_manager.task_stacks.values() -> list of lists of tasks associated with each priority level

i = 0
for task in task_manager.get_next_task(max_priority=0):
    if i > 1:
        break
    print(f"main loop: got task: {task}")
    result = task_manager.execute_task(task)
    print(f"main loop: got result: {result}")
    task_manager.supply_result(result)
    i += 1

/home/jonah/Projects/github.com/konveyor-ecosystem/kai-jonah/kai/service/llm_interfacing/model_provider.py:144: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  self.llm: BaseChatModel = model_class(**model_args)


{'provider': 'ChatOpenAI', 'args': {'model': 'gpt-4'}, 'template': None, 'llama_header': None, 'llm_retries': 5, 'llm_retry_delay': 10.0}


INFO:kai.repo_level_awareness.codeplan:Seed task AnalyzerRuleViolation<priority="0", depth="0", parent="None", children="[]", retry_count="0", max_retries="3", creation_order="0", file="/home/jonah/Projects/github.co...", line="12", column="None", message="Remote EJBs are not supported ...", incident="uri='file:///home/jonah/Projec...", violation="description='Remote EJBs are n...", ruleset="name='quarkus/springboot' desc..."> added to stack.
INFO:kai.repo_level_awareness.codeplan:TaskManager initialized.
INFO:kai.repo_level_awareness.codeplan:Initializing task stacks.
INFO:kai.repo_level_awareness.codeplan:Running validators.
INFO:kai.repo_level_awareness.task_runner.analyzer_lsp.validator:Running analyzer-lsp


/home/jonah/Projects/github.com/konveyor-ecosystem/kai-jonah/notebooks/kai-analyzer-code-plan/tmp-o0ipu7tj/coolstore/src/main/java/com/redhat/coolstore/service/ShippingService.java
AnalyzerRuleViolation<priority="2", depth="0", parent="None", children="[]", retry_count="0", max_retries="3", creation_order="0", file="/home/jonah/Projects/github.co...", line="12", column="None", message="Remote EJBs are not supported ...", incident="uri='file:///home/jonah/Projec...", violation="description='Remote EJBs are n...", ruleset="name='quarkus/springboot' desc...">


INFO:kai.repo_level_awareness.task_runner.analyzer_lsp.validator:2024/10/16 12:56:48 Re-opening moved/deleted file .metadata/.log ...

INFO:kai.repo_level_awareness.task_runner.analyzer_lsp.validator:2024/10/16 12:56:48 Successfully reopened .metadata/.log

INFO:kai.repo_level_awareness.task_runner.analyzer_lsp.validator:analyzer_output.result is not a BaseModel
INFO:kai.repo_level_awareness.task_runner.analyzer_lsp.validator:{'Rulesets': [{'name': 'azure/springboot', 'description': 'Recommend OpenFeign instead of Feign.', 'skipped': ['azure-aws-config-credential-01000', 'azure-aws-config-region-02000', 'azure-aws-config-s3-03000', 'azure-aws-config-sqs-04000', 'azure-aws-config-secret-manager-05000', 'azure-file-system-01000', 'azure-file-system-02000', 'azure-file-system-03000', 'azure-java-version-01000', 'azure-java-version-02000', 'azure-logging-0000', 'azure-os-specific-00001', 'azure-os-specific-00002', 'azure-password-01000', 'eap-to-azure-appservice-certificates-001', 'eap-to-

...

INFO:kai.repo_level_awareness.task_runner.analyzer_lsp.validator:2024/10/16 12:58:17 Re-opening moved/deleted file .metadata/.log ...

INFO:kai.repo_level_awareness.task_runner.analyzer_lsp.validator:2024/10/16 12:58:17 Successfully reopened .metadata/.log

INFO:kai.repo_level_awareness.task_runner.analyzer_lsp.validator:Returning because analyzer_output is JsonRpcError
INFO:kai.repo_level_awareness.task_runner.analyzer_lsp.validator:Error: code=<JsonRpcErrorCode.InternalError: -32603> message='Timeout waiting for response' data=None
INFO:kai.repo_level_awareness.codeplan:Found 1 new tasks from validator <kai.repo_level_awareness.task_runner.analyzer_lsp.validator.AnalyzerLSPStep object at 0x7f47c882d460>
INFO:kai.repo_level_awareness.codeplan:Found 10 new tasks from validator <kai.repo_level_awareness.task_runner.compiler.maven_validator.MavenCompileStep object at 0x7f47c862eba0>
INFO:kai.repo_level_awareness.codeplan:Found 11 new tasks from validators
INFO:kai.repo_level_awareness.co

main loop: got task: PackageDoesNotExistError<priority="0", depth="1", parent="AnalyzerRuleViolation<priority...", children="[]", retry_count="0", max_retries="3", creation_order="152", file="/home/jonah/Projects/github.co...", line="9", column="21", message="package jakarta.ws.rs does not...", details="[]", parse_lines="[ERROR] /home/jonah/Projects/g...", missing_package="jakarta.ws.rs">


INFO:kai.repo_level_awareness.task_runner.dependency.task_runner:got mvn dep response: MavenDependencyResult(encountered_errors=None, modified_files=None, final_answer='', fqdn_response=FQDNResponse(artifact_id='jakarta.ws.rs-api', group_id='jakarta.ws.rs', version='4.0.0'), find_in_pom=FindInPomResponse(start_line=17, end_line=17))
INFO:kai.repo_level_awareness.task_runner.dependency.task_runner:No final answer was given, we need to return with nothing modified. result: MavenDependencyResult(encountered_errors=None, modified_files=None, final_answer='', fqdn_response=FQDNResponse(artifact_id='jakarta.ws.rs-api', group_id='jakarta.ws.rs', version='4.0.0'), find_in_pom=FindInPomResponse(start_line=17, end_line=17))
INFO:kai.repo_level_awareness.codeplan:Supplying result: TaskResult(encountered_errors=[], modified_files=[])
INFO:kai.repo_level_awareness.codeplan:Handling new tasks after processing task: PackageDoesNotExistError<priority="0", depth="1", parent="AnalyzerRuleViolation<prior

main loop: got result: TaskResult(encountered_errors=[], modified_files=[])


INFO:kai.repo_level_awareness.task_runner.analyzer_lsp.validator:2024/10/16 13:02:42 Re-opening moved/deleted file .metadata/.log ...

INFO:kai.repo_level_awareness.task_runner.analyzer_lsp.validator:2024/10/16 13:02:42 Waiting for .metadata/.log to appear...

INFO:kai.repo_level_awareness.task_runner.analyzer_lsp.validator:2024/10/16 13:02:42 Successfully reopened .metadata/.log

INFO:kai.repo_level_awareness.task_runner.analyzer_lsp.validator:Returning because analyzer_output is JsonRpcError
INFO:kai.repo_level_awareness.task_runner.analyzer_lsp.validator:Error: code=<JsonRpcErrorCode.InternalError: -32603> message='Timeout waiting for response' data=None
INFO:kai.repo_level_awareness.codeplan:Found 1 new tasks from validator <kai.repo_level_awareness.task_runner.analyzer_lsp.validator.AnalyzerLSPStep object at 0x7f47c882d460>
INFO:kai.repo_level_awareness.codeplan:Found 10 new tasks from validator <kai.repo_level_awareness.task_runner.compiler.maven_validator.MavenCompileStep object

In [9]:
#verify that the java file has been updated.

import filecmp

diff = rcm.snapshot.diff(rcm.snapshot.parent)

print(diff[1])

r = filecmp.cmp(coolstore_path+"/pom.xml", "./test-data/pom.xml")
print(r)


/home/jonah/Projects/github.com/konveyor-ecosystem/kai-jonah/notebooks/kai-analyzer-code-plan/tmp-o0ipu7tj/coolstore
RepoContextSnapshot(work_tree=PosixPath('/home/jonah/Projects/github.com/konveyor-ecosystem/kai-jonah/notebooks/kai-analyzer-code-plan/tmp-o0ipu7tj/coolstore'), git_dir=PosixPath('/home/jonah/Projects/github.com/konveyor-ecosystem/kai-jonah/notebooks/kai-analyzer-code-plan/tmp-o0ipu7tj/coolstore/.kai/.git-2024-10-16T16:56:10.137262+00:00'), git_sha='277d87bd3212321c315fd7dcaacd19da00b71dcf', parent=None, children=[RepoContextSnapshot(work_tree=PosixPath('/home/jonah/Projects/github.com/konveyor-ecosystem/kai-jonah/notebooks/kai-analyzer-code-plan/tmp-o0ipu7tj/coolstore'), git_dir=PosixPath('/home/jonah/Projects/github.com/konveyor-ecosystem/kai-jonah/notebooks/kai-analyzer-code-plan/tmp-o0ipu7tj/coolstore/.kai/.git-2024-10-16T16:56:10.137262+00:00'), git_sha='d221b7aa11b9f29ee9ca77a8da2b7c3cbfc9485b', parent=..., children=[], spawning_result=[])], spawning_result=None)
R

## Cleanup temp dir

In [5]:
temp_dir.cleanup()